In [128]:
# TODO remove this notebook as soon as ensured that create road layer and farmyard density loayer scripts work correctly

In [114]:
import rasterio
import geopandas as gpd
import sys

sys.path.append("../")
from src.gdal_wrapper import gdal_rasterize_pixel, gdal_align_and_resample

In [14]:
# read in reference grid vector cells
path_to_ref_grid_vector = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\processed\reference_grid\inca_reference_grid_100m_vector_AUT\inca_ref_grid_100m_vector_AUT.shp"
ref_grid_vector = gpd.read_file(path_to_ref_grid_vector)
ref_grid_vector = ref_grid_vector.reset_index()

In [18]:
# read in reference grid specifications
path_to_ref_grid = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\processed\reference_grid\INCA_ref_raster_since_2013_100m.tif"
with rasterio.open(path_to_ref_grid) as src:
    ref_grid_meta = src.profile

In [39]:
# read in farmyard data 
path_to_osm_farmyard = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\processed\farmyard_density_layer\osm_lu_farmyards\osm_lu_farmyards.shp"
farmyards_gdf = gpd.read_file(path_to_osm_farmyard)
farmyards_gdf = farmyards_gdf.to_crs("EPSG:31287")
#farmyards_gdf = farmyards_gdf.reset_index()


In [49]:
ref_raster_polygon_overlay = gpd.overlay(farmyards_gdf, ref_grid_vector)

C:\Users\David\AppData\Local\Temp;\ipykernel_19876\632483798.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:31287
Right CRS: PROJCS["MGI_Austria_Lambert",GEOGCS["GCS_MGI",DATU ...

  ref_raster_polygon_overlay = gpd.overlay(farmyards_gdf, ref_grid_vector)


In [51]:
# Calculate length of road segment in each cell
ref_raster_polygon_overlay["area"] = ref_raster_polygon_overlay.geometry.area
polygon_area_per_cell = ref_raster_polygon_overlay.groupby(
    by="index").agg({"area": "sum"}).reset_index()

In [53]:
polygon_density_layer = ref_grid_vector.merge(
    polygon_area_per_cell, on="index", how="left")


In [98]:
polygon_density_layer_no_nan = polygon_density_layer.dropna()

path_to_farmyard_density_vector = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\processed\test_aoi\test_farmyard_density_vector\test_farmyard_density_vector.shp"
polygon_density_layer_no_nan.to_file(path_to_farmyard_density_vector)

In [111]:
path_to_farmyard_density = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\processed\test_aoi\test_farmyard_density.tif"

extent = ("20003.2793", "220053.2444" ,"720106.901", "620076.3115")


gdal_rasterize_pixel(path_to_farmyard_density_vector, path_to_farmyard_density, "test_farmyard_density_vector", "area", ("7010.0", "4010.0"), "0", extent, "Float32")

In [126]:
# path_to_farmyard_density_resampled = r"C:\Users\David\Documents\ZGIS\Nextcloud_MyFiles\Projects\IGNITE\data\processed\test_aoi\test_farmyard_density_resampled.tif"
resampl_algo = "nearest neighbor"
path_to_farmyard_density_resampled = fr"C:\Users\David\Documents\ZGIS\test_farmyard_density_resampled_{resampl_algo}.tif"

In [127]:
gdal_align_and_resample(path_to_farmyard_density, path_to_farmyard_density_resampled, path_to_ref_grid, resampl_algo)